In [4]:
import pandas as pd
import os
import string
import re
import numpy as np
from nltk.stem import WordNetLemmatizer
import nltk
import pickle
print('NLTK (DOWNLOAD ALL PACKAGES TO PERFORM NLP OPERATION)')

print('UNCOMMENT FOLLOWING LINE To GET NLTK DOWNLOADED')
#nltk.download()
stopword = nltk.corpus.stopwords.words('english')
wordnet_lemmatizer = WordNetLemmatizer()


NLTK (DOWNLOAD ALL PACKAGES TO PERFORM NLP OPERATION)
UNCOMMENT FOLLOWING LINE To GET NLTK DOWNLOADED


In [5]:
def preprocess(df):
    
    df = df[df.columns.drop(list(df.filter(regex='^Cat')))]
    df = df[df['Date'] != '27/06/2001']  #removing the date
    df[(df['Subject'] != 'RE:') & (df['Subject'] != 'FW:') & (df['Subject'] != 'Re:')]  #removing the max same subjects
    del df['Unnamed: 0']
    return df

In [6]:
def clean_text(text):
    text_nopunct = "".join([char for char in text.lower() if char not in string.punctuation])
    tokens = re.split('\W+', text_nopunct) #tokenize
    words_without_stopwords = [word for word in tokens if word not in stopword] #remove stopwords from tokens
    return [wordnet_lemmatizer.lemmatize(word, pos="v") for word in words_without_stopwords]

In [7]:
DATA = 'data' #https://data.world/brianray/enron-email-dataset

FILENAMES = [os.path.join(DATA, filename) for filename in os.listdir(DATA)]
df = pd.read_csv(FILENAMES[0])
df = preprocess(df)

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
df = df[df['Subject'].notna()]
df = df[df['content'].notna()]
df[df['content'].isna()].shape

(0, 15)

In [9]:

dfEmail = df[['Subject', 'content']]
dfEmail.head()

,Subject,content
1,Re:,Traveling to have a business meeting takes the...
2,Re: test,test successful. way to go!!!
4,Re: Hello,Let's shoot for Tuesday at 11:45.
5,Re: Hello,"Greg, How about either next Tuesday or Thursda..."
7,Re: PRC review - phone calls,any morning between 10 and 11:30


Loading saved tfidf vectorizer

In [10]:
content_vecotrizer = pickle.load(open('tfidf_content.pickle', "rb"))
subject_vecotrizer = pickle.load(open('tfidf_subject.pickle', "rb"))
print('Vectorizer Loaded!!')

Vectorizer Loaded!!


In [11]:
dfEmail['Subject_vec'] =  list(subject_vecotrizer.transform(df['Subject']).toarray())

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [12]:
dfEmail.head()

,Subject,content,Subject_vec
1,Re:,Traveling to have a business meeting takes the...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,Re: test,test successful. way to go!!!,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,Re: Hello,Let's shoot for Tuesday at 11:45.,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5,Re: Hello,"Greg, How about either next Tuesday or Thursda...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
7,Re: PRC review - phone calls,any morning between 10 and 11:30,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [13]:
dfEmail['content_vec'] =  list(subject_vecotrizer.transform(df['content']).toarray())

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
